GD-NLP 1. 텍스트 데이터 다루기
====

텍스트 처리성능에 큰 영향을 주는 다양한 텍스트 데이터 전처리 기법을 소개한다. Word나 형태소 레벨의 tokenizer 및 Subword 레벨 tokenizing 기법(BPE, sentencepiece)을 소개하고 사용법을 익힌다.
     
     
1. 전처리 : 자연어의 노이즈 제거
2. 분산표현 : 바나나와 사과의 관계를 어떻게 표현할까?
3. 토큰화 : 그녀는? 그녀+는?
4. 단어의 조각들, Wordpiece Model
5. 토큰에게 의미를 부여하기

# 1-2. 전처리 : 자연어의 노이즈 제거


## 노이즈 유형 (1) 문장부호 : Hi, my name is john.

먼저 노이즈 유형 (1) 문장부호입니다.     
우리는 문장부호를 배웠으니 Hi, 가 Hi 와 , 의 결합인 것을 알지만 컴퓨터는 명시해 주지 않는다면 알파벳에 , 가 포함되어 있다고 생각할 수도 있겠죠? 문장부호를 단어와 분리하면 해결이 되는 상황이기 때문에 문장부호 양쪽에 공백을 추가하는 방법을 취합시다!

In [1]:
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")

    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is john . 


- Python은 replace() 라는 직관적인 함수를 지원해 주기에, 이를 활용하면 풀고자 하는 문제를 쉽게 풀어낼 수 있습니다!
    
단순히 문장부호 양옆에 공백을 추가해 주는 방식은 문제를 야기할 수도 있습니다. 이를테면 "www.semi01sample.com" 라는 구절이 문장에 포함되어 있다면 "www . semi01sample . com" 이라는 요상한 모양새로 변형을 시키겠죠. "Mr.Johnson" 이나 "45,756" 같은 경우도 마찬가집니다.
    
이런 경우는 불가피한 손실로 취급하고 넘어갑니다. 모든 규칙에 대해 변환을 정의해 줄 수는 없기 때문이죠. 게다가 이러한 예외들은 보통 우리가 수행하고자 하는 목적에 큰 영향을 주지 못합니다. "Mr / . / Johnson" 과 "Barack / Hussein / Obama" 는 컴퓨터 입장에서 똑같이 세 단어로 이루어진 이름일 뿐이니까요.

## 노이즈 유형 (2) 대소문자 : First, open the first chapter.

그다음은 노이즈 유형 (2) 대소문자입니다.    
영어에서 발생하는 문제인데, First와 first 는 같은 의미를 갖고 있음에도 컴퓨터는 f와 F를 다르다고 구분 지어 버릴 수 있겠죠. 이를 방지하기 위해 모든 단어를 소문자로 바꾸는 방법을 취하겠습니다.
    
엄연히 말하면 위 예시에서 First와 first는 부사와 한정사로 구분되어 각각의 용도를 배우면 더 좋을 것 같지만, 앞서 언급했듯이 아직은 이른 접근이랍니다!

In [2]:
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


세상에... 이렇게 간단할 수가요? lower() 함수가 있다면 그 반대도 당연히 있겠죠?

In [3]:
sentence = "First, open the first chapter."

print(sentence.upper())

FIRST, OPEN THE FIRST CHAPTER.


## 노이즈 유형 (3) 특수문자 : He is a ten-year-old boy.

마지막인 노이즈 유형 (3) 특수문자입니다. ten-year-old와 seven-year-old, 그 외의 수많은 나이 표현들을 각각의 단어 취급을 해버리는 일이 벌어지겠죠. 그런 불상사를 방지하기 위해 특수문자를 제거하려 합니다. 하지만 무한한 특수문자를 모두 정의하여 제거할 수는 없겠죠? 따라서 우리는 사용할 알파벳과 기호들을 정의해 이를 제외하곤 모두 제거하도록 하겠습니다.

In [4]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


- re 패키지는 정규표현식 사용을 도와주는 패키지입니다. 정규표현식을 사용하면 어떤 문장을 규칙에 따라 일반화하는 것이 굉장히 편해집니다. 대신 처음 접하는 분들께는 기호들이 다소 난해하게 다가오죠...     
    - 정규표현식 정리 : https://hamait.tistory.com/342    
    
지금까지 배운 모든 것을 종합하여 문장을 정제하는 함수로 정의한 후, 다음으로 넘어가도록 하죠!

In [5]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

# 1-4. 토큰화 : 그녀는? 그녀+는?

한 문장에서 단어의 수는 어떻게 정의할 수 있을까요? "그녀는 나와 밥을 먹는다" 라는 문장이 주어지면 공백 기준으로 나누어 1: 그녀는 2: 나와 3: 밥을 4: 먹는다 4개 단어로 이루어졌다고 단정 지을 수 있을까요? 어쩌면 1: 그녀 2: 는 3: 나 4: 와 5: 밥 6: 을 7: 먹는다 처럼 잘게 잘게 쪼개어 7개 단어로 이루어졌다고 할 수도 있겠죠? 그것은 우리가 정의할 토큰화 기법이 결정할 부분이랍니다!
     
문장을 어떤 기준으로 쪼개었을 때, 쪼개진 각 단어들을 토큰(Token) 이라고 부릅니다. 그리고 그 쪼개진 기준이 토큰화(Tokenization) 기법에 의해 정해지죠. 이번 스텝에서는 토큰화의 여러 가지 기법에 대해 배워보도록 하겠습니다.

## 공백 기반 토큰화

자연어의 노이즈를 제거하는 방법 중 하나로 우리는 Hi, 를 Hi와 ,로 나누기 위해 문장부호 양옆에 공백을 추가해 주었습니다. 그것은 이 공백 기반 토큰화를 사용하기 위해서였죠! 당시의 예제 코드를 다시 가져와 공백을 기반으로 토큰화를 진행해 보겠습니다.

In [6]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

- 첫 번째로 배우는 가장 간단한 토큰화 기법이니만큼 사용법도 아주 간단합니다. split() 함수를 호출하기만 하면 자동으로 토큰들의 List로 만들어주죠.
    
단어 사전에 위 단어들이 전부 등재되어 있다면야 썩 나쁘지 않겠죠? 사실상 같은 단어인 days 와 day 가 구분되어 따로 저장이 되는 정도의 문제는 있겠지만, s 를 모두 한 칸 띌 수도 없는 노릇이기에 불가피한 손실로 취급합니다(후에 등장할 방법에서 해결될 문제입니다!).

## 형태소 기반 토큰화

하지만 우리에겐 영어 문장이 아닌 한국어 문장을 처리할 일이 더 많을 것이고, 한국어 문장은 공백 기준으로 토큰화를 했다간 엉망진창의 단어들이 등장하는 것을 알 수 있습니다. 문장부호처럼 "은 / 는 / 이 / 가" 양옆에 공백을 붙이자구요? 글쎄요... 가로 시작하는 단어만 해도 가면, 가위, 가족, 가수... 의도치 않은 변형이 너무나도 많이 일어날 것 같네요!
    
이를 어떻게 해결할 수 있을까요? 정답은 형태소에 있습니다. 어릴 적 국어 시간에 배운 기억이 새록새록 나시나요? 상기시켜드리면 형태소의 정의는 아래와 같습니다.    
- (명사) 뜻을 가진 가장 작은 말의 단위.
    
예를 들어, 오늘도 공부만 한다 라는 문장이 있다면, 오늘, 도, 공부, 만, 한다 로 쪼개지는 것이 바로 형태소죠. 한국어는 이를 활용해 토큰화를 할 수 있습니다!

그럼 실제로 한국어 형태소 분석기를 사용하는 비교 실험을 해 봅시다.
    
우선 한국어 형태소 분석기를 설치하는 과정이 필요하겠군요. 의외로 만만치 않은 작업이 될 수도 있겠습니다만, 앞으로 한국어 자연어처리에 도전하겠다면 이 정도는 필수과정이라고 생각하고 진행해 봅시다.
    
우선 아래와 같이 konlpy 및 Mecab의 설치 여부를 확인해 봅시다.

In [7]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

이번에는 카카오에서 제공하는 Khaiii 형태소 분석기입니다.

- Khaiii 설치      
 sudo apt install cmake   # Khaiii의 빌드 과정을 위해 cmake를 필요로 합니다.    
 pip install torch     # Khaiii는 구동을 위해 파이토치를 필요로 합니다.      
 cd ~/aiffel/text_preprocess    
 git clone https://github.com/modulabs/khaiii_wrapper.git     
 cd ~/aiffel/text_preprocess/khaiii_wrapper/khaiii_pos    
 ./install_khaiii.sh     

In [8]:
import khaiii

api = khaiii.KhaiiiApi()
api.open()

In [9]:
# Khaiii를 konlpy tokenizer처럼 사용하기 위한 wrapper class입니다. 

class Khaiii():
    def pos(self, phrase, flatten=True, join=False):
        """POS tagger.

        :param flatten: If False, preserves eojeols.
        :param join: If True, returns joined sets of morph and tag.

        """
        sentences = phrase.split('\n')
        morphemes = []
        if not sentences:
            return morphemes

        for sentence in sentences:
            for word in api.analyze(sentence):
                result = [(m.lex, m.tag) for m in word.morphs]
                if join:
                    result = ['{}/{}'.format(m.lex, m.tag) for m in word.morphs]

                morphemes.append(result)

        if flatten:
            return sum(morphemes, [])

        return morphemes

설치가 완료되었습니다. 그럼 형태소 분석을 진행해 봅시다. 6개의 형태소 분석기가 다음 문장을 어떻게 해석하는지 비교해 보세요.
- 코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

In [10]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt(),Khaiii()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))
    print()

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]

[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]

[Komoran] 
[('코로나바이러스', 'NNP'), ('는', 'JX'), ('2019', 'S

- 어떤가요? 완벽한 토크나이저는 역시 존재하지 않는 것 같습니다. 저마다 장단점이 존재하는데, 문장을 바꿔 가며 토크나이저마다의 장단점을 정리해 보는 것도 좋은 경험이 될 것입니다.
    
핵심인 단어 코로나바이러스와 우한을 모른다면 제대로 해낼 수 있을 리가 없습니다. 이를 OOV(Out-Of-Vocabulary) 문제라고 합니다. 이처럼 새로 등장한(본 적 없는) 단어에 대해 약한 모습을 보일 수밖에 없는 기법들이기에, 이를 해결하고자 하는 시도들이 있었습니다. 그리고 그것이 우리가 다음 스텝에서 배울, Wordpiece Model이죠!

# 1-5. 단어의 조각들, Wordpiece Model

Wordpiece Model(WPM) 은 우리가 접한 적이 있는 아이디어를 기반으로 만들어졌습니다. 두 단어 preview와 predict를 보면 접두어인 pre가 공통되고 있죠? pre가 들어간 단어는 주로 "미리", "이전의" 와 연계되는 의미를 가지고 있습니다. 컴퓨터도 두 단어를 따로 볼 게 아니라 pre+view와 pre+dict로 본다면 학습을 더 잘 할 수 있지 않을까요?
    
이처럼 한 단어를 여러 개의 Subword의 집합으로 보는 방법이 WPM입니다. WPM의 원리를 알기 전, 먼저 알아야 할 것이 바로 Byte Pair Encoding(BPE) 입니다.

## Byte Pair Encoding(BPE)

데이터에서 가장 많이 등장하는 바이트 쌍(Byte Pair) 을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작합니다.    

모든 단어를 문자(바이트)들의 집합으로 취급하여 자주 등장하는 문자 쌍을 합치면, 접두어나 접미어의 의미를 캐치할 수 있고, 처음 등장하는 단어는 문자(알파벳)들의 조합으로 나타내어 OOV 문제를 완전히 해결할 수 있다는 것이죠!    

- Neural Machine Translation of Rare Words with Subword Units : https://arxiv.org/pdf/1508.07909.pdf    

위 논문은 Python 소스 코드를 함께 제공해 주어 간편하게 실습을 해 볼 수 있습니다. 논문에서 제공해 주는 예제로 동작 방식을 자세히 들여다보죠!

In [11]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merge Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

Merge Vocab: ['es', 'est', 'lo', 'low', 'ne']


- collections.defaultdict() : Return a new dictionary-like object    
    https://docs.python.org/3/library/collections.html#collections.defaultdict
- re.escape() : pattern에서 특수 문자를 이스케이프 처리    
    https://docs.python.org/ko/3/library/re.html#re.escape    
- re.compile() : 정규식 패턴을 정규식 객체로 컴파일    
    https://docs.python.org/ko/3/library/re.html#re.compile
- re.sub(pattern, repl, string) : string에서 겹치지 않는 pattern의 가장 왼쪽 일치를 repl로 치환하여 얻은 문자열을 반환    
    https://docs.python.org/ko/3/library/re.html#re.sub

만일 lowest라는 처음 보는 단어가 등장하더라도, 위 알고리즘을 따르면 어느 정도 의미가 파악된 low와 est의 결합으로 표현할 수 있습니다. 또 BPE의 놀라운 점은 아무리 큰 데이터도 원하는 크기로 OOV 문제없이 사전을 정의할 수 있다는 것입니다. 극단적으로 생각했을 때 알파벳 26개와 특수문자, 문장부호를 아무리 추가해도 100개 이내로 사전을 정의할 수 있죠.(물론 그러면 안 됩니다!!)
    
Embedding 레이어는 단어의 개수 x Embedding 차원 수 의 Weight를 생성하기 때문에 단어의 개수가 줄어드는 것은 곧 메모리의 절약으로 이어집니다. 많은 데이터가 곧 정확도로 이어지기 때문에 이런 기여는 굉장히 의미가 있습니다!
     
하지만 아직도! 완벽하다고는 할 수 없습니다. 만약 수많은 데이터를 사용해 만든 BPE 사전으로 모델을 학습시키고 문장을 생성하게 했다고 합시다. 그게 [i, am, a, b, o, y, a, n, d, you, are, a, gir, l]이라면, 어떤 기준으로 이들을 결합해서 문장을 복원하죠? 몽땅 한꺼번에 합쳤다간 끔찍한 일이 벌어질 것만 같습니다...

## Wordpiece Model(WPM)

이에 구글에서 BPE를 변형해 제안한 알고리즘이 바로 WPM입니다. WPM은 BPE에 대해 두 가지 차별성을 가집니다.
    
1. 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.
2. 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다.
    
첫 번째 문항은 아주 쉬운 내용으로, 앞서 사용한 예문을 빌리면 [_i, _am, _a, _b, o, y, _a, n, d, _you, _are, _a, _gir, l]로 토큰화를 한다는 것입니다. 이렇게 하면 문장을 복원하는 과정이 1) 모든 토큰을 합친 후, 2) 언더바 _를 공백으로 치환으로 마무리되어 간편하죠.    
    
두 번째 문항은 다소 난해하게 다가올 수도 있습니다.     
- 논문 3,4절 참고 : https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf    

구글의 SentencePiece 라이브러리를 통해 고성능의 BPE를 사용할 수 있습니다! SentencePiece에는 전처리 과정도 포함되어 있어서, 데이터를 따로 정제할 필요가 없어 간편하기까지 합니다.    
- google/sentencepiece : https://github.com/google/sentencepiece